<a href="https://colab.research.google.com/github/pakka79/pakka79/blob/main/Daily_OHLC_N_Engulfing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libary===================================================#
import FinanceDataReader as fdr
from pykrx import stock
import warnings
import pandas as pd
import numpy as np
import talib as ta
import datetime as dt
import pytz
warnings.filterwarnings("ignore")
#Libary===================================================#

KoreaTime = pytz.timezone('Asia/Seoul')
From_Today=1000
today = dt.datetime.now()
previouday=today-dt.timedelta(days=From_Today)
Today=today.strftime("%Y%m%d")
Previuosday=previouday.strftime("%Y%m%d")

From_Today_Dart=1
TodayDart=today.strftime("%Y-%m-%d")
previoudaydart=today-dt.timedelta(days=From_Today_Dart)
PreviuosdayDart=previoudaydart.strftime("%Y-%m-%d")

#TICKER1===================================================#
KRX=["KOSPI","KOSDAQ"]
kospi=fdr.StockListing("KOSPI")
kosdaq=fdr.StockListing("KOSDAQ")
krx=pd.concat([kospi,kosdaq])
krx[['Symbol', 'Market', 'Name', 'Sector', 'Industry']]
krx.rename(columns={'Symbol':'KRX_TICKER_LIST'},inplace=True)
krx=krx[['KRX_TICKER_LIST', 'Market', 'Name', 'Sector', 'Industry']]
krx=krx.dropna(subset=['Sector'])
#TICKER1===================================================#

#===========================================================================================#
for i in krx['KRX_TICKER_LIST']:
    try:
        df = stock.get_market_ohlcv(Previuosday, Today, i)

        df['회사명'] = stock.get_market_ticker_name(i)
        df['티커'] = i
        df = df[["티커", "회사명", "시가", "고가", "저가", "종가", "거래량"]]
        df["PCT"]=round(df["종가"].pct_change(),4)

        #Build VWAP
        df['VWAP']=(df['저가'] + df['종가'] + df['고가']).div(3).values
        df['TP_n_V']=df['VWAP'].values*df['거래량'].values
        df['VWAP_5']=(df.TP_n_V.rolling(window=5).sum().values)/df['거래량'].rolling(window=5).sum().values
        df['VWAP_10']=(df.TP_n_V.rolling(window=10).sum().values)/df['거래량'].rolling(window=10).sum().values
        df['VWAP_20']=(df.TP_n_V.rolling(window=20).sum().values)/df['거래량'].rolling(window=20).sum().values
        df['VWAP_60']=(df.TP_n_V.rolling(window=60).sum().values)/df['거래량'].rolling(window=60).sum().values

        # Build Volume.mean
        df['Vol_30'] = round(df.거래량.rolling(30).mean())
        df['Vol_60'] = round(df.거래량.rolling(60).mean())

        # Engulfing 
        df['Engulfing'] = ta.CDLENGULFING(df['시가'], df['고가'], df['저가'], df['종가'])
        
        # Shooting Star
        df['SHOOTINGSTAR'] = ta.CDLSHOOTINGSTAR(df['시가'], df['고가'], df['저가'], df['종가'])

        # Build ATR
        df['ATR'] = round(ta.ATR(df['고가'], df['저가'], df['종가'], timeperiod=20))

        # Build Bollinger Band
        def Build_Bollinger_Band(df):
            df['SMA_5'] = round(df.종가.rolling(5).mean())
            df['SMA_20'] = round(df.종가.rolling(20).mean())
            df['stddev'] = round(df.종가.rolling(5).std())
            df['Upper'] = round(df.SMA_5 + 2.5 * df.stddev)
            df['Lower'] = round(df.SMA_5 - 2.5 * df.stddev)
            df['rsi'] = round(ta.RSI(df['종가'], timeperiod=14))
        Build_Bollinger_Band(df)

        df=df[['티커', '회사명', '시가', '고가', '저가', '종가','PCT','거래량', 'VWAP','VWAP_5', 'VWAP_10', 'VWAP_20', 'VWAP_60', 'Vol_30', 'Vol_60', 'Engulfing','SHOOTINGSTAR' ,'ATR','SMA_5', 'SMA_20', 'stddev', 'Upper', 'Lower', 'rsi']]

        print("{}".format(df.회사명[0]), df[-1:])
        
    except:
        pass
#===========================================================================================#